### Housing Prices Analytics
#### Topic: Analyze previous year's home price in Los Angeles
    Katherine Le and Miranda Rendon


In [ ]:
# import packages
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# load datasets
prices = pd.read_csv("MedianPricesofExistingDetachedHomesHistoricalData.csv")
timeOnMarket = pd.read_csv("MedianTimeonMarketofExistingDetachedHomesHistoricalData.csv")
unsoldInv = pd.read_csv("UnsoldInventoryIndexofExistingDetachedHomesHistoricalData.csv")

In [7]:
# get LA data from each set in a new df
housingData = pd.DataFrame({
    'Mon-Yr' : prices['Mon-Yr'],
    'Median Price' : prices['Los Angeles'],
    'Median Time on Market' : timeOnMarket['LA'],
    'Unsold Inventory' : unsoldInv['Los Angeles']

})

# remove incomplete 2024 data
housingData = housingData.iloc[:-8]

# save cleaned data into new file
housingData.to_csv('LAHousinData.csv', index = False)